In [1]:
import pandas as pd
from scipy.stats import chi2_contingency
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
df = pd.read_excel('dataset.xlsx')

In [ ]:
col_columns = [df.columns[3], df.columns[4]]
row_columns = col_columns + df.columns[5:].tolist()

In [52]:
def create_crosstab_with_significance(df, row_columns, col_columns):
    full_table = []

    for row_col in row_columns:
        combined_section = pd.DataFrame()
        
        row_totals = df[row_col].value_counts().sort_index()
        row_totals = pd.DataFrame(row_totals)
        row_totals.index = pd.MultiIndex.from_product([[row_col], row_totals.index])
        row_totals.columns = ['Total']

        for col_col in col_columns:
            crosstab = pd.crosstab(index=df[row_col], columns=df[col_col], margins=False)
            total_by_col = crosstab.sum(axis=0)
            grand_total = crosstab.values.sum()

            crosstab_percent = crosstab.div(crosstab.sum(axis=1), axis=0) * 100
            crosstab_percent = crosstab_percent.astype(object)
            crosstab_percent.index = pd.MultiIndex.from_product([[row_col], crosstab.index])
            multi_index_columns = [(col_col, val) for val in crosstab.columns]
            crosstab_percent.columns = pd.MultiIndex.from_tuples(multi_index_columns)

            if row_col == col_col:
                for index in crosstab.index:
                    for column in crosstab.columns:
                        crosstab_percent.loc[(row_col, index), (col_col, column)] = f"{crosstab_percent.loc[(row_col, index), (col_col, column)]:.2f}%"
            else:
                for index in crosstab.index:
                    for column in crosstab.columns:
                        observed = crosstab.loc[index, column]
                        row_total = row_totals.loc[(row_col, index), 'Total']
                        col_total = total_by_col[column]

                        contingency_table = [
                            [observed, row_total - observed],
                            [col_total, grand_total - col_total]
                        ]

                        expected_counts = [
                            (row_total * col_total) / grand_total,
                            (row_total * (grand_total - col_total)) / grand_total,
                            ((grand_total - row_total) * col_total) / grand_total,
                            ((grand_total - row_total) * (grand_total - col_total)) / grand_total
                        ]

                        if all(count >= 5 for count in expected_counts):
                            chi2, p, _, _ = chi2_contingency(contingency_table)

                            if p < 0.05:
                                if observed > expected_counts[0]:
                                    crosstab_percent.loc[(row_col, index), (col_col, column)] = f"{crosstab_percent.loc[(row_col, index), (col_col, column)]:.2f}%H"
                                else:
                                    crosstab_percent.loc[(row_col, index), (col_col, column)] = f"{crosstab_percent.loc[(row_col, index), (col_col, column)]:.2f}%L"
                            else:
                                crosstab_percent.loc[(row_col, index), (col_col, column)] = f"{crosstab_percent.loc[(row_col, index), (col_col, column)]:.2f}%"
                        else:
                            crosstab_percent.loc[(row_col, index), (col_col, column)] = f"{crosstab_percent.loc[(row_col, index), (col_col, column)]:.2f}%"

            if combined_section.empty:
                combined_section = crosstab_percent
            else:
                combined_section = pd.concat([combined_section, crosstab_percent], axis=1)

        combined_section.insert(0, ('Total', ''), (row_totals['Total'] / grand_total * 100).map(lambda x: f"{x:.2f}%" if pd.notnull(x) else x) )

        full_table.append(combined_section)

    combined_crosstab = pd.concat(full_table, axis=0)
    return combined_crosstab

In [ ]:
def highlight_cells(val):
    if 'H' in str(val):
        return 'background-color: #e63946'
    elif 'L' in str(val):
        return 'background-color: #457b9d'
    return ''

df_crosstab_sig=create_crosstab_with_significance(df, row_columns, col_columns)
df_crosstab_sig = df_crosstab_sig.style.map(highlight_cells)


df_crosstab_sig